In [ ]:
!pip install huggingface-hub

In [ ]:
!pip install transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import transformers
import re
import torch
import json
import numpy as np

In [ ]:
# Kívánt nyelvi modell letöltése.

from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer_l = AutoTokenizer.from_pretrained("xlm-roberta-large")

model_l = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

In [ ]:
'''
regex matchelés
'''

count = 0
used_words_w_idx = list()
delete_words = list()

regex = re.compile("^[A-ZÁÉÚŐÓŰÜÖÍa-záéúűőóüöí0-9._▁—\-\/\\()\"':,;<>!@#$%^&*|+=[\]{}`~?]*$")

for word in list(tokenizer_l.vocab.items()):
    if not regex.match(word[0]):
        # print(word[0], word[1])
        # count += 1
        delete_words.append(word)
    else:
        used_words_w_idx.append(word)
    # if count == 10:
        # break

In [ ]:
# A tokenizer kezelése, hogy elérhessük majd felülírjuk a megfelelő subtokeneket.

tokenizer_l._tokenizer.save("roberta_large_tokenizer.json", True)
json_roberta_large_tokenizer = json.load(open("roberta_large_tokenizer.json", "r"))
json_list = json_roberta_large_tokenizer["model"]["vocab"]

In [ ]:
json_dict = {i[0]: i[1] for i in json_list}
json_dict_copy = json_dict.copy()

print(len(json_dict))

# A nem megfelelő tokenek kidobása.

for delete in delete_words:
    # print(delete[0])
    json_dict_copy.pop(delete[0])

In [ ]:
len(json_dict_copy)

In [ ]:
# A módosított vocab-bal felül írjuk a tokenizert.

json_list_modified = [[i[0], i[1]] for i in json_dict_copy.items()]
json_roberta_large_tokenizer["model"]["vocab"] = json_list_modified

with open('roberta_large_tokenizer_modified.json', 'w') as f:
    json.dump(json_roberta_large_tokenizer, f)

In [ ]:
# Végül létrehozunk egy új tokenizert a szűrt tokenekkel.

from transformers import PreTrainedTokenizerFast

modified_tokenizer_l = PreTrainedTokenizerFast(tokenizer_file="roberta_large_tokenizer_modified.json")

In [ ]:
# Következő 3 esetben kikeressük a megfelelő tokeneket és a hozzátartozó vektorokat, majd pedig készítünk egy embeddinget és egy decodert.

numpy_array_embedding = np.zeros([91842, 1024])
pos = ""

for id in used_words_w_idx:
    pos = modified_tokenizer_l.encode(id[0])
    numpy_array_embedding[pos] = model_l.state_dict()['roberta.embeddings.word_embeddings.weight'][id[1]]
    # break

In [ ]:
numpy_array_decoder = np.zeros([91842, 1024])
pos = ""

for id in used_words_w_idx:
    pos = modified_tokenizer_l.encode(id[0])
    numpy_array_decoder[pos] = model_l.state_dict()["lm_head.decoder.weight"][id[1]]
    # break

In [ ]:
numpy_array_bias = np.zeros([91842])
pos = ""

for id in used_words_w_idx:
    pos = modified_tokenizer_l.encode(id[0])
    numpy_array_bias[pos] = model_l.state_dict()["lm_head.decoder.bias"][id[1]]
    # break

In [ ]:
# Ismét a következő esetekben a már elkészített embeddinget és a decodert átadjuk a modellnek, felülírva a régit.

embedding_tensor = torch.Tensor(numpy_array_embedding)

embedding_own = torch.nn.Embedding(91842, 1024)
embedding_own.state_dict()['weight'][:] = embedding_tensor

print(len(model_l.state_dict()['roberta.embeddings.word_embeddings.weight']))

model_l.roberta.embeddings.word_embeddings = embedding_own

print(len(model_l.state_dict()['roberta.embeddings.word_embeddings.weight']))

In [ ]:
decoder_tensor = torch.Tensor(numpy_array_decoder)

decoder_own = torch.nn.Linear(1024, 91842)
decoder_own.state_dict()['weight'][:] = decoder_tensor

print(len(model_l.state_dict()['lm_head.decoder.weight']))

model_l.lm_head.decoder = decoder_own

print(len(model_l.state_dict()['lm_head.decoder.weight']))

In [ ]:
bias_tensor = torch.Tensor(numpy_array_bias)

decoder_own.state_dict()['bias'][:] = bias_tensor

print(len(model_l.lm_head.bias))

model_l.lm_head.bias = decoder_own.bias

print(len(model_l.lm_head.bias))

##Ellenőrzés és minimális igazítás:

In [ ]:
model_l

In [ ]:
modified_tokenizer_l

In [ ]:
modified_tokenizer_l.model_max_length = 512

In [ ]:
modified_tokenizer_l.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': "<mask>"})

In [ ]:
modified_tokenizer_l

##HF-re feltöltést:

In [ ]:
model_l.push_to_hub('large2')

In [ ]:
modified_tokenizer_l.push_to_hub('large2')

# Huggingface-n át kell írni manuálisan a vocab méretet a config.json-ben!

# Check model

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

gszabo_token = AutoTokenizer.from_pretrained("gszabo/large2")

gszabo_model = AutoModelForMaskedLM.from_pretrained("gszabo/large2")

In [ ]:
gszabo_token

In [ ]:
gszabo_model